In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.spatial.distance as dist
import matplotlib as mpl

import iblofunmatch.inter as ibfm
import os

from navground import core, sim

output_dir = "output"
os.makedirs("output", exist_ok=True)

In [ ]:
length = 8.0
num_steps = 5000
width=1.0
yaml = f"""
steps: 3000
time_step: 0.1
save_directory: ''
record_pose: true
record_twist: true
scenario:
  type: Corridor
  length: {length}
  width: {width} 
  groups:
    -
      type: thymio
      number: 26
      radius: 0.08
      control_period: 0.1
      speed_tolerance: 0.02
      kinematics:
        type: 2WDiff
        wheel_axis: 0.094
        max_speed: 0.166
      behavior:
        type: HL
        optimal_speed: 0.12
        horizon: 5.0
        safety_margin: 0.02
      state_estimation:
        type: Bounded
        range: 5.0
"""
experiment = sim.load_experiment(yaml)
experiment.run()

In [ ]:
run = experiment.runs[0]
ps = run.poses[:,:,[0,1]]
twists = run.twists[:,:,:2] # ignore angular speeds

Take also into account the velocities at each instance.

In [ ]:
weight = 8

In [ ]:
# def corridor_distance(points, length):
#     dist_0 = dist.squareform(dist.pdist(points, "minkowski", p=1))
#     shift_points = np.array(points) # make a copy
#     left_pts_idx = shift_points[:,0] < length/2
#     shift_points[left_pts_idx] += [length,0]
#     dist_1 = dist.squareform(dist.pdist(shift_points, "minkowski", p=1))
#     return np.minimum(dist_0, dist_1)

In [ ]:
def trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length):
    assert(len(positions)>0)
    assert(len(positions)==len(velocities))
    positions_velocities_list = []
    for idx, points in enumerate(positions):
        positions_velocities_list.append(np.hstack((points, velocities[idx]*weight)))
    distances_list = []
    for idx, points_vel in enumerate(positions_velocities_list):
        # Compare accross all different times
        # for j in range(idx+1):
        #     points_vel_compare = positions_velocities_list[j]
        #     dist_0 = dist.cdist(points_vel, points_vel_compare, "minkowski", p=2)
        #     shift_points_vel_compare = np.array(points_vel_compare)
        #     shift_points_vel_compare[shift_points_vel_compare[:,0]<length/2]+=[length, 0, 0, 0]
        #     shift_points_vel = np.array(points_vel)
        #     shift_points_vel[shift_points_vel[:,0]<length/2]+=[length, 0, 0, 0]
        #     dist_1 = dist.cdist(shift_points_vel, shift_points_vel_compare, "minkowski", p=2)
        #     distances_list.append(np.minimum(dist_0, dist_1))
        # # end for 
        # Compare trajectories at same time
        points_vel_compare = positions_velocities_list[idx]
        dist_0 = dist.cdist(points_vel, points_vel_compare, "minkowski", p=2)
        shift_points_vel_compare = np.array(points_vel_compare)
        shift_points_vel_compare[shift_points_vel_compare[:,0]<length/2]+=[length, 0, 0, 0]
        shift_points_vel = np.array(points_vel)
        shift_points_vel[shift_points_vel[:,0]<length/2]+=[length, 0, 0, 0]
        dist_1 = dist.cdist(shift_points_vel, shift_points_vel_compare, "minkowski", p=2)
        distances_list.append(np.minimum(dist_0, dist_1))
    # end for
    distances_arr = np.array(distances_list)
    return np.min(distances_arr, axis=0)

In [ ]:
dist_result = trajectory_corridor_distance_weighted_velocities(ps[:5], twists[:5], weight, length)

In [ ]:
def distances_corridor_weighted_velocities(points, velocities, weight, length):
    points_velocities = np.hstack((points, velocities*weight))
    dist_0 = dist.squareform(dist.pdist(points_velocities, "minkowski", p=2))
    shift_points_vels = np.array(points_velocities) # make a copy
    left_pts_idx = shift_points_vels[:,0] < length/2
    shift_points_vels[left_pts_idx] += [length,0,0,0]
    dist_1 = dist.squareform(dist.pdist(shift_points_vels, "minkowski", p=2))
    return np.minimum(dist_0, dist_1)

In [ ]:
distances_corridor_weighted_velocities(ps[100], twists[100], 2, length).shape

In [ ]:
def compute_divergence_vector(X, Y, vel_X, vel_Y, weight):
    idx_S = list(range(int(X.shape[0])))
    # Compute distane matrices
    # Dist_X = corridor_distance(X, length)
    # Dist_X = Dist_X + distances_weighted_velocities(vel_X, weight)
    Dist_X = distances_corridor_weighted_velocities(X, vel_X, weight, length)
    # Dist_Y = corridor_distance(Y, length)
    # Dist_Y = Dist_Y + distances_weighted_velocities(vel_Y, weight)
    Dist_Y = distances_corridor_weighted_velocities(Y, vel_Y, weight, length)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    # Compute matching
    ibfm_out = [
        ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1),
        ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    ]
    # Compute induced matchings
    matching_XZ = ibfm_out[0]["induced_matching_0"]
    matching_YZ = ibfm_out[1]["induced_matching_0"]
    composition_XY = [matching_YZ.index(i) for i in matching_XZ]
    endpoints_0 = np.array(ibfm_out[0]["S_barcode_0"][:,1])
    endpoints_1 = np.array(ibfm_out[1]["S_barcode_0"][:,1])
    endpoints_1 = endpoints_1[composition_XY]
    return  endpoints_1-endpoints_0, ibfm_out[0]["X_barcode_0"][:,1]

In [ ]:
divergence_list = []
Z_barcodes_list = []
steps_list = list(range(0, 2000, 5))
for start_step in steps_list:
    X = ps[start_step]
    Y = ps[start_step + 30]
    divergence, Z_barcode = compute_divergence_vector(X, Y, twists[start_step], twists[start_step + 30], weight)
    divergence_list.append(divergence)
    Z_barcodes_list.append(Z_barcode)

divergence_arr = np.array(divergence_list).transpose()
Z_barcodes_arr = np.array(Z_barcodes_list).transpose()

Next, we plot divergence vectors over time.

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(divergence_arr)
mapable = ax.imshow(divergence_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
print(mapable)
print(np.max(divergence_arr))
ax.set_title("corridor")
plt.colorbar(mapable)
plt.tight_layout()
plt.savefig("corridor_divergence_diag_hist.png")

Let us plot the persistence diagrams over time.

In [ ]:
def plot_matching_diagram_trajectories(positions, velocities, weight, length, ax, color="blue"):
    # Compute X, Z barcodes and matching
    Dist_X = distances_corridor_weighted_velocities(positions[0], velocities[0], weight, length)
    Dist_Z = trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length)
    idx_S = list(range(Dist_X.shape[0]))
    # Compute matching from X to Z
    ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    # Plot 0 persistence diagram of matching 
    match_diagram = []
    for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
        idx_match = ibfm_out["induced_matching_0"][idx]
        bar_Z = ibfm_out["X_barcode_0"][idx_match]
        match_diagram.append([bar_X[1], bar_Z[1]])
    # end for
    match_diagram = np.array(match_diagram)
    # Plot matching diagram
    ax.scatter(match_diagram[:,0], match_diagram[:,1], color=color)
    ax.plot([0,np.max(match_diagram)*1.1], [0,np.max(match_diagram)*1.1], color="gray")

In [ ]:
ps[start_step:start_step+30].shape

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
steplist = list(range(0, 500, 3))
for idx, start_step in enumerate(steplist):
    plot_matching_diagram_trajectories(ps[start_step:start_step+100], twists[start_step:start_step+100], weight, length, ax, color=mpl.colormaps["GnBu"](idx/len(steplist)))


In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
start_step=1000
shift_time = 100
plot_matching_diagram_trajectories(ps[start_step:start_step+shift_time], twists[start_step:start_step+shift_time], weight, length, ax, color="blue")
# get data
positions = ps[start_step:start_step+shift_time]
velocities = twists[start_step:start_step+shift_time]
Dist_X = distances_corridor_weighted_velocities(positions[0], velocities[0], weight, length)
Dist_Y = trajectory_corridor_distance_weighted_velocities(positions, velocities, weight, length)
Dist_Z = np.minimum(Dist_X, Dist_Y)
idx_S = list(range(Dist_X.shape[0]))
# Compute matching from X to Z
ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
# put persistence pairs together
match_diagram = []
for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
    idx_match = ibfm_out["induced_matching_0"][idx]
    bar_Z = ibfm_out["X_barcode_0"][idx_match]
    match_diagram.append([bar_X[1], bar_Z[1]])
# end for
match_diagram = np.array(match_diagram)

print(np.array(match_diagram))
print(ibfm_out["S_reps_0"])

In [ ]:
def plot_sequence(X_list, ax, mark_points=[]):
    # Plot figure
    X_old = X_list[0]
    ax.scatter(X_old[:,0], X_old[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"](1/(len(X_list)+1)), zorder=2)
    for idx, X in enumerate(X_list[1:]):
        ax.scatter(X[:,0], X[:,1], s=20, marker="o", color=mpl.colormaps["RdBu"]((idx+1)/(len(X_list)+1)), zorder=2, label="X")
        # for edge in zip(X, X_old):
        #     edge_pts = np.array(edge)
        #     ax.plot(edge_pts[:,0], edge_pts[:,1], color="gray", zorder=1)
        if len(mark_points)>0:
            mark_X = X[mark_points]
            ax.scatter(mark_X[:,0], mark_X[:,1], s=20, marker="+", color="red", zorder=3)
        X_old = X
    #end for 

In [ ]:
start_step=900
X_seq = ps[list(range(start_step, start_step+shift_time+1, 10))]
len(X_seq)
fig, ax = plt.subplots(figsize=(10, 5))
ax.set_aspect("equal")
plot_sequence(X_seq, ax, mark_points=[0])

In [ ]:
def plot_matching_diagram(X, Y, vel_X, vel_Y, idx_S, weight, length, ax, color="blue"):
    # Compute X, Z barcodes and matching
    Dist_X = distances_corridor_weighted_velocities(X, vel_X, weight, length)
    Dist_Y = distances_corridor_weighted_velocities(Y, vel_Y, weight, length)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    # Compute matching from X to Z
    ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    # Plot 0 persistence diagram of matching 
    match_diagram = []
    for idx, bar_X in enumerate(ibfm_out["S_barcode_0"]):
        idx_match = ibfm_out["induced_matching_0"][idx]
        bar_Z = ibfm_out["X_barcode_0"][idx_match]
        match_diagram.append([bar_X[1], bar_Z[1]])
    # end for
    match_diagram = np.array(match_diagram)
    # Plot matching diagram
    ax.scatter(match_diagram[:,0], match_diagram[:,1], color=color)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
steplist = list(range(600, 650, 2))
for idx, start_step in enumerate(steplist):
    X = ps[start_step]
    Y = ps[start_step + 30]
    vel_X = twists[start_step]
    vel_Y = twists[start_step+30]
    idx_S = list(range(X.shape[0]))
    plot_matching_diagram(X, Y, vel_X, vel_Y, idx_S, weight, length, ax, color=mpl.colormaps["GnBu"](idx/len(steplist)))

In [ ]:
fig, ax = plt.subplots(figsize=(4,4))
start_step=550
X = ps[start_step]
Y = ps[start_step + 30]
vel_X = twists[start_step]
vel_Y = twists[start_step+30]
idx_S = list(range(X.shape[0]))
plot_matching_diagram(X, Y, vel_X, vel_Y, idx_S, weight, length, ax, color=mpl.colormaps["GnBu"](idx/len(steplist)))
Dist_X = distances_corridor_weighted_velocities(X, vel_X, weight, length)
Dist_Y = distances_corridor_weighted_velocities(Y, vel_Y, weight, length)
Dist_Z = np.minimum(Dist_X, Dist_Y)
# Compute matching from X to Z
ibfm_out = ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
print(ibfm_out["X_barcode_0"][-5:])
print(ibfm_out["X_reps_0"][-5:])

In [ ]:
X_seq = ps[list(range(350,380, 10))]
len(X_seq)
fig, ax = plt.subplots(figsize=(10, 5))
plot_sequence(X_seq, ax, mark_points=[15])

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(divergence_arr)
mapable = ax.imshow(Z_barcodes_arr, aspect="auto", vmax=vmax, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
ax.set_title("Z_barcode_corridor")
plt.colorbar(mapable)
plt.tight_layout()

In [ ]:
cumulative_list = []
for j, divergence in enumerate(divergence_arr.transpose()):
    cumulative = []
    for i in range(divergence_arr.shape[0]):
        cumulative.append(np.sum(divergence[:i+1]))
    cumulative_list.append(cumulative)
cumulative_arr = np.array(cumulative_list).transpose()

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(cumulative_arr)
vmin = np.min(cumulative_arr)
mapable = ax.imshow(cumulative_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
ax.set_title("cumulative")
plt.colorbar(mapable)
plt.tight_layout()

In [ ]:
y_max = np.max(Z_barcodes_arr)
y_min = np.min(Z_barcodes_arr)
y_range = np.linspace(y_min, y_max, 30)
combinations_list = []
for y in y_range:
    combinations_row = [] 
    for j, column in enumerate(Z_barcodes_arr.transpose()):
        idx = np.sum(Z_barcodes_arr[:,j] < y)-1
        if idx < 0:
            combinations_row.append(0)
        else:
            combinations_row.append(cumulative_arr[idx,j])
    combinations_list.append(combinations_row)
combinations_arr = np.array(combinations_list)

In [ ]:
fig, ax = plt.subplots(figsize=(10,2))
vmax = np.max(combinations_arr)
vmin = np.min(combinations_arr)
mapable = ax.imshow(combinations_arr, aspect="auto", vmax=vmax, vmin=vmin, extent=(steps_list[0], steps_list[-1], 0, X.shape[0]))
ax.set_title("combinations")
plt.colorbar(mapable)
plt.tight_layout()

In [ ]:
def plot_two_timesteps(X, Y, ax, X_col="blue", Y_col="red"):
    # Plot figure
    ax.scatter(X[:,0], X[:,1], s=20, marker="s", c=X_col, zorder=2, label="X")
    ax.scatter(Y[:,0], Y[:,1], s=23, marker="x", c=Y_col, zorder=2, label="Y")
    for edge in zip(X, Y):
        edge_pts = np.array(edge)
        ax.plot(edge_pts[:,0], edge_pts[:,1], c="gray", zorder=1)

def plot_divergence_diagram(X, Y, vel_X, vel_Y, weight, length, ax):
    idx_S = list(range(int(X.shape[0])))
    # Compute distane matrices
    Dist_X = distances_corridor_weighted_velocities(X, vel_X, weight, length)
    Dist_Y = distances_corridor_weighted_velocities(Y, vel_Y, weight, length)
    Dist_Z = np.minimum(Dist_X, Dist_Y)
    # Compute induced matchings
    ibfm_out = [
        ibfm.get_IBloFunMatch_subset(Dist_X, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1),
        ibfm.get_IBloFunMatch_subset(Dist_Y, Dist_Z, idx_S, output_dir, max_rad=-1, num_it=1, store_0_pm=True, points=False, max_dim=1)
    ]
    # Divergence diagrams 
    ibfm.plot_XYZ_matching_0(ibfm_out, ax)
    ax.set_xlim([-4,4])
    # print persistence divergence 
    matching_XZ = ibfm_out[0]["induced_matching_0"]
    matching_YZ = ibfm_out[1]["induced_matching_0"]
    composition_XY = [matching_YZ.index(i) for i in matching_XZ]
    endpoints_0 = np.array(ibfm_out[0]["S_barcode_0"][:,1])
    endpoints_1 = np.array(ibfm_out[1]["S_barcode_0"][:,1])
    endpoints_1 = endpoints_1[composition_XY]
    persistence_divergence = np.sum(np.sqrt((endpoints_0-endpoints_1)**2))
    # print(np.abs(endpoints_0-endpoints_1))
    # print(persistence_divergence)
    return ibfm_out

In [ ]:
start_step = 1150
X = ps[start_step]
Y = ps[start_step + 20]
vel_X = twists[start_step]
vel_Y = twists[start_step + 20]
X_len = X.shape[0]-1
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(10, 5))
# Interpret diagram via points
plot_two_timesteps(X, Y, ax[0,0])
plot_divergence_diagram(X, Y, vel_X, vel_Y, weight, length, ax[0,1])
# change timestep 
start_step = start_step+60
X = ps[start_step]
Y = ps[start_step + 20]
vel_X = twists[start_step]
vel_Y = twists[start_step + 20]
plot_two_timesteps(X, Y, ax[1,0])
plot_divergence_diagram(X, Y, vel_X, vel_Y, weight, length, ax[1,1])
plt.tight_layout()
plt.savefig("corridor_exp_divergence.png")